In [13]:
!pip install langchain

In [14]:
!pip install langchain_community

In [15]:
!pip install unstructured

In [16]:
!pip install llama-cpp-python==0.1.78

In [17]:
!pip install "unstructured[docx]"

In [18]:
!pip install sentence-transformers==2.2.2

In [19]:
!pip install faiss-gpu

In [20]:
!pip install InstructorEmbedding

In [21]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [22]:
template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [23]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q2_K.bin

--2024-08-03 13:04:14--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q2_K.bin
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.55, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/45833e0b59c8fe80676c664f556031fc411da8856e0716ac7b8ed201b7221c08?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q2_K.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q2_K.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1722949454&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjk0OTQ1NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQxOWIzOTc1MmM4ZD

In [24]:
import langchain
langchain.verbose = False
llm = LlamaCpp(
    model_path="llama-2-7b-chat.ggmlv3.q2_K.bin",
    n_ctx=6000,
    n_gpu_layers=512,
    n_batch=30,
    callback_manager=callback_manager,
    temperature = 0.9,
    max_tokens = 4095,
    n_parts=1,

)

llm_chain = LLMChain(prompt=prompt, llm=llm)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [25]:
loader = UnstructuredFileLoader("API Security Risks.docx")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# embedding engine
hf_embedding = HuggingFaceInstructEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 0.4.0. An updated version of the class exists in the langchain-unstructured package and should be used instead. To use it run `pip install -U langchain-unstructured` and import as `from langchain_unstructured import UnstructuredLoader`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [26]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [27]:
db = FAISS.from_documents(docs, embed_model)

# save embeddings in local directory
db.save_local("aigrace")

# load from local
db = FAISS.load_local("aigrace/", embeddings=embed_model,allow_dangerous_deserialization=True)

In [28]:
query = "What is Separation of development?"
search = db.similarity_search(query, k=2)

In [29]:
template = """Use the document to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(input_variables=["context", "question"], template= template)
final_prompt = prompt.format(question=query, context=search)

In [30]:
llm_chain.run(final_prompt)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Separation of Development (SoD) refers to the practice of separating the development, testing, and production environments in a way that each environment is isolated from the others and has limited access to resources and data. This approach helps prevent unauthorized access or malicious changes to sensitive information during development, testing, or production.

In the context provided, SoD refers to the practice of separating the development, testing, and production environments in a way that each environment is isolated from the others and has limited access to resources and data. This approach helps prevent unauthorized access or malicious changes to sensitive information during development, testing, or production.

'Separation of Development (SoD) refers to the practice of separating the development, testing, and production environments in a way that each environment is isolated from the others and has limited access to resources and data. This approach helps prevent unauthorized access or malicious changes to sensitive information during development, testing, or production.\n\nIn the context provided, SoD refers to the practice of separating the development, testing, and production environments in a way that each environment is isolated from the others and has limited access to resources and data. This approach helps prevent unauthorized access or malicious changes to sensitive information during development, testing, or production.'

In [ ]:
import sys
from transformers import AutoModelWithLMHead, AutoTokenizer
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import Chroma
template = """You are cyber security assisstant. Use the document to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(input_variables=["context", "question"], template= template)

qa_chain = ConversationalRetrievalChain.from_llm(
llm, retriever=db.as_retriever(search_kwargs={'k': 2}),combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True
)
chat_history = []
while True:
    query = input('Prompt: ')
    if query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))


Prompt: how to secure api?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit
